# **Tutorial 4** - Geophysics (Seismology)

In this tutorial we will learn how to

 1. Prepare **gridded data**: [`pygmt.xyz2grd`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grd2xyz.html)
 2. Create a **contour map**: [`pygmt.Figure.grdcontour`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.Figure.grdcontour.html)
 3. Create a **profile plot**: [`pygmt.grdproject`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grdproject.html) and [`pygmt.grdtrack`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grdtrack.html)

-----
This tutorial is part of the AGU2024 annual meeting GMT/PyGMT pre-conference workshop (PREWS9) **Mastering Geospatial Visualizations with GMT/PyGMT**
- Conference: https://agu.confex.com/agu/agu24/meetingapp.cgi/Session/226736
- GitHub: https://github.com/GenericMappingTools/agu24workshop
- Website: https://www.generic-mapping-tools.org/agu24workshop
- Recommended version: PyGMT v0.13.0 with GMT 6.5.0

## 0. General stuff

Import the required packages. Besides [`PyGMT`](https://www.pygmt.org/v0.13.0) we also use [`NumPy`](https://numpy.org/doc/stable/).

In [ ]:
import pygmt
import numpy as np

## 1. Prepare gridded data: [`pygmt.xyz2grd`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grd2xyz.html)


## 2. Create a contour map: [`pygmt.Figure.grdcontour`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.Figure.grdcontour.html)
Convert grids or images to contours and plot them on maps

you need to define:
1. `grid`: Accessing the remote datasets or providing your dataset as ``xarray.DataArray``
2. `level`: Specify the contour intervals to generate; for example, value of 1000 means plotting contour lines at every 1000 m.
3. `annotation`: Annotated contour levels
4. `limit`: Draw contours below low or above high, e.g, [-4000, 0] means drawing contour lines below sea level and above -4000 m.

In [ ]:
fig = pygmt.Figure()

grid = pygmt.datasets.load_earth_relief(resolution="05m", region=[-90, -60, -40, 0])
cpt = pygmt.makecpt(cmap='earth', series=[-7000, 7000], continuous=True)

fig.grdimage(grid=grid, cmap=cpt, projection="M10c", frame=['a10f5'])

fig.grdcontour(grid=grid, annotation=2000, levels=1000, limit=[-4000, 0], pen='0.5p,white')

fig.colorbar(frame=["x+lElevation", "y+lm", 'a2000f1000'])
fig.show()

## 3. Create a profile plot: [`pygmt.grdproject`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grdproject.html) and [`pygmt.grdtrack`](https://www.pygmt.org/v0.13.0/api/generated/pygmt.grdtrack.html)


### need to add 

In [ ]:
start_x = -85
start_y = -20
end_x = -62
end_y= -20

In [ ]:

# Choose a survey line
fig.plot(
    x=[start_x, end_x],  # Longitude in degrees East
    y=[start_y, end_y],  # Latitude in degrees North
    # Draw a 2-points thick red dashed line for the survey line
    pen="1p,red",
)

fig.text(
    x=[start_x, end_x],  # Longitude in degrees East
    y=[start_y, end_y],  # Latitude in degrees North
    text=["A", "A'"],
    offset="0c/0.3c", 
    font="10p,1,white",
    fill='red')


fig.show()

### need to add 

In [ ]:
# Generate points along a great circle corresponding to the survey line
# and store them in a pandas.DataFrame
track_df = pygmt.project(
    center=f"{start_x}/{start_y}",  # Start point of survey line (longitude/latitude)
    endpoint=f"{end_x}/{end_y}",  # End point of survey line (longitude/latitude)
    generate="10",  # Output data in steps of 10 km 
    unit=True
)


In [ ]:
track_df.head()

NOTE: in this DataFrame, 

**r**: 

**s**: 

**p**: 


## need to add

In [ ]:
# Extract the elevation at the generated points from the downloaded grid
# and add it as new column "elevation" to the pandas.DataFrame
track_df = pygmt.grdtrack(
    grid=grid,
    points=track_df,
    newcolname="elevation",
)


In [ ]:
track_df

Note: after applying ``grdtrack``, the new column is **elevation** to indicate the elevation variation along the track

In [ ]:
print(min(track_df.elevation),max(track_df.elevation))

In [ ]:
fig.shift_origin(yshift="15.7c")

fig.basemap(
    region=[0, max(track_df.p), -7500, 5000],  # x_min, x_max, y_min, y_max
    # Cartesian projection with a width of 12 centimeters and
    # a height of 3 centimeters
    projection="X10c/2c",
    # Add annotations ("a") and ticks ("f") as well as labels ("+l")
    # at the west or left and south or bottom sides ("WSrt")
    frame=["WSrt", "xa500f250+lDistance (km)", "ya2500+lElevation (m)"],
)

fig.text(
    x=[0, max(track_df.p)],
    y=[5000, 5000],
    text=["A", "A'"],
    offset='0c/0.3c',
    no_clip=True,  # Do not clip text that fall outside the plot bounds
    font="12p",  # Use a font size of 10 points
)

fig.show()



In [ ]:
# Plot water masses
fig.plot(
    x=[0, max(track_df.p)],
    y=[0, 0],
    fill="lightblue",  # Fill the polygon in "lightblue"
    # Draw a 0.25-points thick black solid outline
    pen="0.25p,black,solid",
    close="+y-8000",  # Force closed polygon
)

# Plot elevation along the survey line
fig.plot(
    x=track_df.p,
    y=track_df.elevation,
    fill="gray",  # Fill the polygon in "gray"
    # Draw a 1-point thick black solid outline
    pen="1p,black,solid",
    close="+y-8000",  # Force closed polygon
)

fig.show()

## 4. Add additional features

## 5. Additional comments

Some helpful and interesting aspects:

- Use suitable colormaps for your data: [**Scientific colourmaps by Fabio Crameri**](https://www.fabiocrameri.ch/colourmaps/), see also the publications [Crameri et al. 2024](https://doi.org/10.1002/cpz1.1126) and [Crameri et al. 2020](https://doi.org/10.1038/s41467-020-19160-7)